### Extract apply link domains

The following code extract the domain of apply links. The extraction is used to decide which apply boards are mostly used. I'm aiming to automate the application process at these job boards.

__DON'T REMOVE__

In [9]:
import pandas as pd
from src.db import DB
import re

db = DB("results/jobs.sqlite")
# https://something.domain.com/blahblahblah -> domain.com
pat = re.compile(r"https?:\/\/(.*\.)?(.*\..*?)\/.*")
q = """
SELECT d.id, d.apply_link, c.name FROM details as d 
LEFT JOIN company as c ON d.company_id = c.id
WHERE apply_link IS NOT NULL;
"""
db.cursor.execute(q)
raw = db.cursor.fetchall()
df = pd.DataFrame(columns=["id","apply_link","company_name"],data=raw)
df["apply_board"] = df["apply_link"].apply(lambda lnk: pat.findall(lnk)[0][-1])
df['apply_board'].value_counts()

OperationalError: unable to open database file

In [ ]:
from dotenv import load_dotenv
from src.driver import Driver
from src.scanner import Scanner
from time import sleep
# from selenium.webdriver.support import expected_conditions as ec
load_dotenv()
driver = Driver(load_timeout=-1, driver_logging=False,debug_address="localhost:9222")
driver.driver.refresh()
sleep(2)
form = driver.driver.find_element("xpath",)
scanner = Scanner(driver.driver,"//div[contains(@data-automation-id,'Page')]")
scanner.run()

In [3]:
from dotenv import load_dotenv
from src.driver import Driver
from src.scanner import Scanner
from src.routines.myworkday import Routine
from time import sleep
# from selenium.webdriver.support import expected_conditions as ec
load_dotenv()
driver = Driver(load_timeout=-1, driver_logging=False,debug_address="localhost:9222")
# driver.driver.implicitly_wait(10)
routine = Routine(driver.driver,"https://td.wd3.myworkdayjobs.com/en-US/TD_Bank_Careers/job/Toronto%2C-Ontario/Sr-Software-Engineer--ETrading--ION_R_1346079/apply/applyManually")
# routine.goto_url().pause(2).get_creds().sign_in().pause(3).refresh()#.pause(5).my_information().pause(1).find_click_submit_btn().pause(5).my_experience()
# routine.find_click_submit_btn().pause(5).my_experience(selected_sections=["files"])

## Scan all form elements

In [ ]:
from dotenv import load_dotenv
from src.driver import Driver
from src.scanner import Scanner
from selenium.webdriver import Keys, ActionChains
load_dotenv()
driver = Driver(load_timeout=-1, driver_logging=False,debug_address="localhost:9222")
scanner = Scanner(driver.driver,"div[contains(@data-automation-id,'Page')]")
# scanner.refresh_for_form()
results = []
for el in scanner.form_elements():
    el_type = scanner.find_input_type(el)
    match el_type: 
        case "radio":
            text, choices = scanner.find_radio_text(el)
            results.append({
                "is_group": False,
                "type": el_type,
                "element": el,
                "text": text,
                "choices": choices
            })
        case "checkbox":
            text, choices = scanner.find_checkbox_text(el)
            if text:
                results.append({
                    "is_group": True,
                    "type": el_type,
                    "element": el,
                    "text": text,
                    "choices": choices
                })
            else:
                results.append({
                    "is_group": False,
                    "type": el_type,
                    "element": el,
                    "text": text,
                    "choices": None
                })
        case _:
            text = scanner.find_text(el),
            results.append({
                "is_group": False,
                "type": el_type,
                "element": el,
                "text": text,
                "choices": None
            })
results

In [1]:
from dotenv import load_dotenv
from src.driver import Driver
from src.scanner import Scanner
from selenium.webdriver import Keys, ActionChains
from src.routines.myworkday import Routine
load_dotenv()
driver = Driver(load_timeout=-1, driver_logging=False,debug_address="localhost:9222")
scanner = Scanner(driver.driver,"div[contains(@data-automation-id,'Page')]")
routine = Routine(driver.driver,"https://td.wd3.myworkdayjobs.com/en-US/TD_Bank_Careers/job/Toronto%2C-Ontario/Sr-Software-Engineer--ETrading--ION_R_1346079/apply/applyManually")
# routine.refresh().pause(5).find_click_submit_btn().pause(5).find_click_submit_btn()
for _ in range(6):
    res = scanner.find_dropdown_text()
    scanner.reset_dropdown()
    print(res)
    scanner.press_key()

['No, I am not 18 years or older', 'Yes, I am 18 years or older']
['No, I do not meet this requirement', 'Yes, I meet this requirement']
['No, I am not', 'Yes, I am']
['No, I will not be able to meet this requirement', 'Yes, I understand this requirement']
['Yes, I currently hold or have previously held an insurance license', 'No, I have not']
['No, my insurance license has never been refused, revoked or suspended', 'Yes, my insurance license is or has previously been refused, revoked or suspended']


In [4]:
routine.refresh().pause(5).find_click_submit_btn().pause(5).find_click_submit_btn()

In [19]:
inp = scanner.active_element().find_element("xpath","following::input")
scanner.driver.execute_script("arguments[0].removeAttribute('value')",inp)
scanner.driver.execute_script("arguments[0].innerText='select one'",scanner.active_element())

In [2]:
el = scanner.active_element()

In [3]:
scanner.driver.execute_script("arguments[0].focus()",el)

In [4]:
scanner.active_element().click()